# CEV分析ノートブック（複数結果対応版）

resultsフォルダ内の全ての実行結果からCEVを計算・比較表示

In [1]:
import pickle
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from olg.cev.cev_analysis import run_cev_analysis
from olg.transition.setting import TransitionSetting
from olg.transition.forward import solve_forward_transition


# その他のmatplotlib設定
plt.rcParams["figure.figsize"] = (14, 10)
plt.rcParams["font.size"] = 11
plt.rcParams["axes.unicode_minus"] = False  # マイナス記号の文字化け防止

In [2]:
def parse_result_prefix(filename):
    """
    結果ファイルのプレフィックスから政策パラメータを抽出
    例: 'psi_50to25_by25T' → {'psi_ini': 0.5, 'psi_fin': 0.25, 'TT': 25}
    """
    # psi_XXtoYY_byZZT パターンにマッチ
    pattern = r"psi_(\d+(?:\.\d+)?)to(\d+(?:\.\d+)?)_by(\d+)T"
    match = re.search(pattern, filename)

    if match:
        psi_ini_str, psi_fin_str, TT_str = match.groups()

        # 特殊ケース：0は実際は非常に小さな値
        psi_ini = float(psi_ini_str) / 100 if psi_ini_str != "0" else 0.5
        psi_fin = float(psi_fin_str) / 100 if psi_fin_str != "0" else 0.000000001
        TT = int(TT_str)

        return {
            "psi_ini": psi_ini,
            "psi_fin": psi_fin,
            "TT": TT,
            "label": f"ψ: {psi_ini:.2f}→{psi_fin:.3f} ({TT}T)",
        }
    return None


def scan_results_folder():
    """
    resultsフォルダをスキャンして利用可能な結果ファイルを取得
    """
    if not os.path.exists("results"):
        print("resultsフォルダが見つかりません")
        return []

    results = []

    # results直下のフォルダを探索
    for item in os.listdir("results"):
        folder_path = os.path.join("results", item)
        if os.path.isdir(folder_path) and item.startswith("results_"):
            # フォルダ内のpklファイルを探索
            pkl_files = [f for f in os.listdir(folder_path) if f.endswith("_full.pkl")]

            for pkl_file in pkl_files:
                prefix = pkl_file.replace("_full.pkl", "")
                params = parse_result_prefix(prefix)

                if params:
                    results.append(
                        {
                            "folder": item,
                            "file": pkl_file,
                            "prefix": prefix,
                            "params": params,
                            "path": os.path.join(folder_path, pkl_file),
                        }
                    )

    # パラメータでソート
    results.sort(key=lambda x: (x["params"]["psi_ini"], x["params"]["psi_fin"]))
    return results


# 利用可能な結果をスキャン
available_results = scan_results_folder()
print(f"見つかった結果: {len(available_results)}個\\n")

for i, result in enumerate(available_results):
    print(f"{i+1}. {result['params']['label']} - {result['folder']}/{result['file']}")

if not available_results:
    print("⚠️ 結果ファイルが見つかりません。main.pyを実行して結果を生成してください。")

見つかった結果: 3個\n
1. ψ: 0.50→0.250 (25T) - results_psi_50to25_by25T/psi_50to25_by25T_full.pkl
2. ψ: 0.50→0.750 (25T) - results_psi_50to75_by25T/psi_50to75_by25T_full.pkl
3. ψ: 0.50→1.000 (25T) - results_psi_50to100_by25T/psi_50to100_by25T_full.pkl


In [3]:
available_results

[{'folder': 'results_psi_50to25_by25T',
  'file': 'psi_50to25_by25T_full.pkl',
  'prefix': 'psi_50to25_by25T',
  'params': {'psi_ini': 0.5,
   'psi_fin': 0.25,
   'TT': 25,
   'label': 'ψ: 0.50→0.250 (25T)'},
  'path': 'results/results_psi_50to25_by25T/psi_50to25_by25T_full.pkl'},
 {'folder': 'results_psi_50to75_by25T',
  'file': 'psi_50to75_by25T_full.pkl',
  'prefix': 'psi_50to75_by25T',
  'params': {'psi_ini': 0.5,
   'psi_fin': 0.75,
   'TT': 25,
   'label': 'ψ: 0.50→0.750 (25T)'},
  'path': 'results/results_psi_50to75_by25T/psi_50to75_by25T_full.pkl'},
 {'folder': 'results_psi_50to100_by25T',
  'file': 'psi_50to100_by25T_full.pkl',
  'prefix': 'psi_50to100_by25T',
  'params': {'psi_ini': 0.5,
   'psi_fin': 1.0,
   'TT': 25,
   'label': 'ψ: 0.50→1.000 (25T)'},
  'path': 'results/results_psi_50to100_by25T/psi_50to100_by25T_full.pkl'}]

In [ ]:
def load_and_compute_cev(result_info):
    """
    単一の結果ファイルを読み込んでCEVを計算
    """
    print(f"\\n=== {result_info['params']['label']} の処理開始 ===")

    # 結果ファイルを読み込み
    with open(result_info["path"], "rb") as f:
        results = pickle.load(f)

    initial_result, final_result, K_path, opt_indexes, aprimes, value_functions = (
        results
    )

    # TransitionSettingを再構築
    params = result_info["params"]
    tr_setting = TransitionSetting(
        NT=100, TT=params["TT"], psi_ini=params["psi_ini"], psi_fin=params["psi_fin"]
    )

    print(
        f"設定: NT=100, TT={params['TT']}, ψ_ini={params['psi_ini']:.3f}, ψ_fin={params['psi_fin']:.6f}"
    )

    # 分布パスを計算
    print("分布パス計算中...")
    mu_dists = solve_forward_transition(
        tr_setting, initial_result.hp, opt_indexes, initial_result.mu_dist_box
    )

    # CEV計算
    print("CEV計算中...")
    cev_results = run_cev_analysis(
        tr_setting=tr_setting,
        initial_result=initial_result,
        value_functions=value_functions,
        mu_dists=mu_dists,
        plot_period=0,  # 第1期
        plot_age=0,  # 20歳
        show_plots=False,  # 個別プロットは無効
        save_plots=False,
    )

    return {
        "result_info": result_info,
        "cev_avg": cev_results["cev_avg"],
        "initial_result": initial_result,
        "tr_setting": tr_setting,
    }


# 全ての結果に対してCEV計算を実行
all_cev_results = []

if available_results:
    print("=== 全結果のCEV計算開始 ===")

    for result_info in available_results:
        try:
            cev_result = load_and_compute_cev(result_info)
            all_cev_results.append(cev_result)
            print(f"✅ {result_info['params']['label']} 完了")
        except Exception as e:
            print(f"❌ {result_info['params']['label']} でエラー: {e}")

    print(f"\\n=== CEV計算完了: {len(all_cev_results)}個の結果 ===")
else:
    print("スキップ: 利用可能な結果がありません")

=== 全結果のCEV計算開始 ===
\n=== ψ: 0.50→0.250 (25T) の処理開始 ===
❌ ψ: 0.50→0.250 (25T) でエラー: No module named 'olg_solver'
\n=== ψ: 0.50→0.750 (25T) の処理開始 ===
❌ ψ: 0.50→0.750 (25T) でエラー: No module named 'olg_solver'
\n=== ψ: 0.50→1.000 (25T) の処理開始 ===
❌ ψ: 0.50→1.000 (25T) でエラー: No module named 'olg_solver'
\n=== CEV計算完了: 0個の結果 ===


In [4]:
# CEV結果の比較可視化
def plot_cev_comparison_across_policies(cev_results_list, period=0, age_idx=0):
    """
    複数の政策に対するCEVを比較プロット
    """
    if not cev_results_list:
        print("プロット対象の結果がありません")
        return

    # 年齢軸の設定（最初の結果から）
    first_result = cev_results_list[0]
    if first_result["initial_result"] is not None:
        ages = np.arange(20, 20 + first_result["initial_result"].hp.NJ)
        l_grid = first_result["initial_result"].hp.l_grid
    else:
        # デフォルト値
        NJ = first_result["cev_avg"].shape[1]
        ages = np.arange(20, 20 + NJ)
        l_grid = np.array([0.7, 1.3])

    # カラーマップの設定
    colors = plt.cm.viridis(np.linspace(0, 1, len(cev_results_list)))

    # 1. 年齢別CEV比較（第1期）
    plt.figure(figsize=(16, 12))

    # サブプロット1: Low skill
    plt.subplot(2, 2, 1)
    for i, cev_result in enumerate(cev_results_list):
        cev_avg = cev_result["cev_avg"]
        label = cev_result["result_info"]["params"]["label"]
        plt.plot(
            ages, cev_avg[period, :, 0], "--", color=colors[i], linewidth=2, alpha=0.8
        )

    plt.axhline(y=0, color="gray", linestyle=":", alpha=0.7)
    plt.xlabel("Age")
    plt.ylabel("CEV")
    plt.title(f"CEV Comparison: Low Skill (Period {period+1})")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True, alpha=0.3)

    # サブプロット2: High skill
    plt.subplot(2, 2, 2)
    for i, cev_result in enumerate(cev_results_list):
        cev_avg = cev_result["cev_avg"]
        label = cev_result["result_info"]["params"]["label"]
        plt.plot(
            ages,
            cev_avg[period, :, 1],
            "-",
            color=colors[i],
            linewidth=2,
            alpha=0.8,
            label=label,
        )

    plt.axhline(y=0, color="gray", linestyle=":", alpha=0.7)
    plt.xlabel("Age")
    plt.ylabel("CEV")
    plt.title(f"CEV Comparison: High Skill (Period {period+1})")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True, alpha=0.3)

    # サブプロット3: コホート別（特定年齢での時系列）
    plt.subplot(2, 2, 3)
    cohorts = np.arange(1, cev_results_list[0]["cev_avg"].shape[0] + 1)

    for i, cev_result in enumerate(cev_results_list):
        cev_avg = cev_result["cev_avg"]
        label = cev_result["result_info"]["params"]["label"]
        # High skill のコホート別CEV
        plt.plot(
            cohorts,
            cev_avg[:, age_idx, 1],
            "-",
            color=colors[i],
            linewidth=2,
            alpha=0.8,
            label=f"{label} (High)",
        )
        # Low skill のコホート別CEV
        plt.plot(
            cohorts,
            cev_avg[:, age_idx, 0],
            "--",
            color=colors[i],
            linewidth=1.5,
            alpha=0.6,
            label=f"{label} (Low)",
        )

    plt.axhline(y=0, color="gray", linestyle=":", alpha=0.7)
    plt.xlabel("Cohort (Period)")
    plt.ylabel("CEV")
    plt.title(f"CEV Time Series: Age {20+age_idx}")
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True, alpha=0.3)


# 全ての結果に対して比較プロットを作成
if all_cev_results:
    print("=== CEV比較プロット作成 ===")
    plot_cev_comparison_across_policies(all_cev_results, period=0, age_idx=0)

    # 各政策の詳細統計を表示
    print("\\n=== 政策別CEV統計（第1期） ===")
    for cev_result in all_cev_results:
        cev_avg = cev_result["cev_avg"]
        params = cev_result["result_info"]["params"]

        print(f"\\n{params['label']}:")
        for i_l in range(cev_avg.shape[2]):
            skill_name = "Low" if i_l == 0 else "High"
            period_0_values = cev_avg[0, :, i_l]
            mean_cev = np.mean(period_0_values)
            std_cev = np.std(period_0_values)
            min_cev = np.min(period_0_values)
            max_cev = np.max(period_0_values)

            print(
                f"  {skill_name} skill: Mean={mean_cev:.4f}, Std={std_cev:.4f}, Range=[{min_cev:.4f}, {max_cev:.4f}]"
            )
else:
    print("プロット対象の結果がありません")

プロット対象の結果がありません


In [5]:
def calculate_factor_prices_path(tr_setting, K_path, initial_setting):
    """移行過程の要素価格を計算"""
    L = initial_setting.Njw / initial_setting.NJ  # 労働供給

    r_path = np.zeros(tr_setting.NT)
    w_path = np.zeros(tr_setting.NT)

    for t in range(tr_setting.NT):
        K_t = K_path[t]
        r_path[t] = (
            initial_setting.alpha * (K_t / L) ** (initial_setting.alpha - 1)
            - initial_setting.delta
        )
        w_path[t] = (1 - initial_setting.alpha) * (K_t / L) ** initial_setting.alpha

    return r_path, w_path


def plot_capital_path(tr_setting, K_path, initial_result, final_result):
    """資本パスと金利パスをプロット"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # 資本パス
    periods = np.arange(1, tr_setting.NT + 1)
    ax1.plot(periods, K_path, "b-", linewidth=2, label="Capital Path")
    ax1.axhline(
        y=initial_result.K,
        color="red",
        linestyle="--",
        alpha=0.7,
        label=f"Initial SS (K={initial_result.K:.3f})",
    )
    ax1.axhline(
        y=final_result.K,
        color="green",
        linestyle="--",
        alpha=0.7,
        label=f"Final SS (K={final_result.K:.3f})",
    )
    ax1.set_xlabel("Period")
    ax1.set_ylabel("Capital Stock")
    ax1.set_title("Capital Stock Transition Path")
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 金利パス
    r_path, _ = calculate_factor_prices_path(tr_setting, K_path, initial_result.hp)
    ax2.plot(periods, r_path, "r-", linewidth=2, label="Interest Rate Path")
    ax2.axhline(
        y=initial_result.r,
        color="red",
        linestyle="--",
        alpha=0.7,
        label=f"Initial SS (r={initial_result.r:.4f})",
    )
    ax2.axhline(
        y=final_result.r,
        color="green",
        linestyle="--",
        alpha=0.7,
        label=f"Final SS (r={final_result.r:.4f})",
    )
    ax2.set_xlabel("Period")
    ax2.set_ylabel("Interest Rate")
    ax2.set_title("Interest Rate Transition Path")
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


plot_capital_path(tr_setting, K_path, initial_result, final_result)

NameError: name 'tr_setting' is not defined

In [ ]:
available_results

[{'folder': 'results_20251108_114241',
  'file': 'psi_50to25_by25T_full.pkl',
  'prefix': 'psi_50to25_by25T',
  'params': {'psi_ini': 0.5,
   'psi_fin': 0.25,
   'TT': 25,
   'label': 'ψ: 0.50→0.250 (25T)'},
  'path': 'results/results_20251108_114241/psi_50to25_by25T_full.pkl'},
 {'folder': 'results_20251108_141245',
  'file': 'psi_50to75_by25T_full.pkl',
  'prefix': 'psi_50to75_by25T',
  'params': {'psi_ini': 0.5,
   'psi_fin': 0.75,
   'TT': 25,
   'label': 'ψ: 0.50→0.750 (25T)'},
  'path': 'results/results_20251108_141245/psi_50to75_by25T_full.pkl'},
 {'folder': 'results_20251108_14580000',
  'file': 'psi_50to100_by25T_full.pkl',
  'prefix': 'psi_50to100_by25T',
  'params': {'psi_ini': 0.5,
   'psi_fin': 1.0,
   'TT': 25,
   'label': 'ψ: 0.50→1.000 (25T)'},
  'path': 'results/results_20251108_14580000/psi_50to100_by25T_full.pkl'}]

In [ ]:
all_cev_results[0]

{'result_info': {'folder': 'results_20251108_114241',
  'file': 'psi_50to25_by25T_full.pkl',
  'prefix': 'psi_50to25_by25T',
  'params': {'psi_ini': 0.5,
   'psi_fin': 0.25,
   'TT': 25,
   'label': 'ψ: 0.50→0.250 (25T)'},
  'path': 'results/results_20251108_114241/psi_50to25_by25T_full.pkl'},
 'cev_avg': array([[[ 2.15290783e-02,  1.74473301e-02],
         [ 1.92636948e-02,  1.40722769e-02],
         [ 1.66675953e-02,  1.09477883e-02],
         ...,
         [-8.27509702e-03, -8.27459451e-03],
         [-4.10827931e-03, -4.10812810e-03],
         [-9.68365602e-07, -9.68332648e-07]],
 
        [[ 2.15290783e-02,  1.74473301e-02],
         [ 1.86380316e-02,  1.17338687e-02],
         [ 1.56703033e-02,  8.67571176e-03],
         ...,
         [-4.92394397e-03, -4.92361427e-03],
         [-9.49182908e-05, -9.49096278e-05],
         [ 3.14170433e-03,  3.14158561e-03]],
 
        [[ 2.49216566e-02,  2.07928225e-02],
         [ 2.20802077e-02,  1.51875588e-02],
         [ 1.80608387e-02,  9.